# **Research on 2020 SPACs**

## **DataSet description**

Tickers.csv - The master list of all the tickers that are being analyzed. Following are the column definitions.
  1. spac_symbol - Ticker symbol of spac stock.
  2. merge_symbol - Ticker symbol of merged company.
  3. name - name of the SPAC company
  4. news_or_rumor_date - Rumor or confirmed news on when the potential merger was announced to general public.
  5. price_before_news - Price in dollars before the news was broken to public.
  6. merge_date - Date on which the merger happened. N/A means merger hasn't happened yet.
  7. trust_value_dollars - The trust value of the company. In case of dilution, this is the value that shareholders get back.
  8. merger_company_value_billions - Proposed value of the merging company in billion dollars.
  9. spac_target_sector- Target sector of the SPAC as per their SEC filing.
  10. merging_company_sector - Sector of the company that was announced to be merged with.

TickerName.csv (eg. HOFV.csv)- The file that has historic cost and volume of each tickers.
  1. Date - Date of price and volume action
  2. Open - Price of stock in dollars during opening of the day at 9:00 AM EST.
  3. High - Day's highest price in dollars.
  4. Low- Day's low price in dollars.
  5. Close - Day's close price in dollars.
  5. Adj Close - Day's adjusted close price in dollars.
  6. Volume - Day's volume count


## **Section 1: Initial Data setup and Cleanup**


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from statistics import mean

In [ ]:
datadir = 'data/'

Import the tickers into dataframe

In [ ]:
spac_tickers_dir = datadir + 'Tickers.csv'
spac_tickers_list = pd.read_csv(spac_tickers_dir)
spac_tickers_list.head(5)

,spac_symbol,merge_symbol,name,news_or_rumor_date,price_before_news,merge_date,spac_market_cap,trust_value_dollars,merger_company_value_billions,spac_target_sector,merging_company_sector
0,GPAQ,HOFV,Gordon Pointe Acquisition Corp,5/26/2020,11.00,7/1/2020,31060000,10.81,0.4935,FinTech,Entertainment
1,ARYA,IMTX,ARYA Sciences Acquisition Corp,3/17/2020,10.58,7/1/2020,148698960,10.34,1.4000,Healthcare,Biotechnology
2,PAAC,LGHL,Proficient Alpha Acquisition Corp,3/10/2020,10.11,6/16/2020,4586096,10.18,0.1250,FinTech,Asset Management
3,NEBU,LPRO,Nebula Acquisition Corp,5/18/2020,10.30,6/10/2020,250000000,10.26,1.8000,Technology,Fintech
4,VTIQ,NKLA,VectoIQ Acquisition Corp,3/3/2020,11.64,6/3/2020,200000000,10.33,3.5000,Automotive,EV


## **Section 2: Reusable functions for the notebook**

In [ ]:

#Function to get a ticker's historice stock data
def get_symbol_df(symbol=None, days = 50, index_column = ''):
    
    if index_column=="":
      df = pd.DataFrame(pd.read_csv(datadir + symbol + '.csv', parse_dates=True))[:days]
      df.reset_index(inplace=True)
    else:
      df = pd.DataFrame(pd.read_csv(datadir + symbol + '.csv', index_col=index_column, parse_dates=True))[:days]

    return df.dropna() #Make sure to clean up invalid data

#Test the function with a sample ticker
#hof_df = get_symbol_df('HOFV', 30,'Date')
#hof_df.head(3)
hof_df = get_symbol_df('BWMC', 300)
print(hof_df)

     index        Date    Open    High    Low   Close  Adj Close    Volume
0        0  2019-08-20  10.040  10.050  10.04  10.050     10.050  309100.0
1        1  2019-08-21  10.050  10.050  10.05  10.050     10.050       0.0
2        2  2019-08-22  10.050  10.050  10.05  10.050     10.050       0.0
3        3  2019-08-23  10.040  10.057  10.03  10.040     10.040  112200.0
4        4  2019-08-26  10.050  10.050  10.05  10.050     10.050   20000.0
..     ...         ...     ...     ...    ...     ...        ...       ...
99      99  2020-01-10  10.180  10.190  10.15  10.155     10.155   12100.0
100    100  2020-01-13  10.160  10.190  10.15  10.179     10.179   17600.0
101    101  2020-01-14  10.179  10.179  10.15  10.150     10.150    1400.0
102    102  2020-01-15  10.182  10.190  10.15  10.150     10.150    9800.0
103    103  2020-01-16  10.160  10.172  10.16  10.170     10.170    1900.0

[104 rows x 8 columns]


In [ ]:
def get_annotations(xLabel, yLable):
  return [ dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text= xLabel,
            xref="paper",
            yref="paper"
        ),
        dict(
            x=-0.04,
            y=0.5,
            showarrow=False,
            text= yLable,
            textangle=-90,
            xref="paper",
            yref="paper"
        )]

## **Section 3: Price action for 30 days after Merger announcement**

In this section lets learn about the change in price of stock after a merger announcement news is floated. Lets focus on first 30 days after the date of announcement

In [ ]:
fig = go.Figure()

for ticker in spac_tickers_list['spac_symbol']:
  ticker_data = get_symbol_df(symbol=ticker, days=30)
  fig.add_scatter(x=ticker_data['index'], y=ticker_data['High'], hovertemplate="Day: %{x} and High:%{y}$", name = ticker)

fig.update_layout(title_text = 'Price volatility from the day merger news is announced',
                  annotations=get_annotations('Days since news', 'Price in $'))

fig.show()


**Inference**: 
1. SPACs that had more than 50% spike on day 1 of announcement had high likelihood for further growth or maintained the price for first 30 days.

2. SPAcs that never raised above 10% after first 13 days were likely not to raise any further and declined below it's trust value in some cases

3. Also, for those that had inital price hike, they tend to decline within 6 days before the price consolidates on a new base

## **Section 4: Price and Volume action after merger announcement**

In the first part of section lets compare the volume and price action in order to identify top winners and in the subsequent part, lets analyse the day to day volume action of winners after the announcement date to identify the first day volume threshold.

### Average volume and average price action of each tickers

In [ ]:
#Conslide the data for all the tickers
volume=[]
tickers=[]
prize=[]

for ticker in spac_tickers_list['spac_symbol']:
  ticker_data = get_symbol_df(symbol=ticker, days=300)
  tickers.append(ticker)
  volume.append(mean(ticker_data['Volume'].values.tolist()))
  prize.append(mean(ticker_data['Close'].values.tolist()))


In [ ]:
data= [tickers, prize, volume]

fig = go.Figure(data=go.Heatmap(
                   x=tickers,
                   y=prize,
                   z=volume,
                   colorbar=dict(title='Volume'),
                   hovertemplate="Ticker: %{x}<br>Average prize:%{y}$<br>Average Volume = %{z}",
                   hoverongaps = False))

fig.update_layout(title_text = 'Average volume and prize heat map',
                  annotations=get_annotations('SPAC Tickers', 'Price in $'))
fig.show()

**Inference:**

1. It is intuitive to say that the higher volume would result in higher price
2. Above heatmap suggests that an average volume on 1 million and above is needed to stay at a average prize of $12 and above 

### Volume and price action analysis for Winners

In the previous section we concluded that if there is no good price action within first 13 days, then its unlikely to have any better price action later. Also, in this section we identified winners and laggards based on the volume action. Now lets analysis the top 4 winners for their volume in first 30 days. Based on the heat map following tickers are identified as winners.
1. Shll
2. VTIQ
3. SPAQ and
4. FMCI

In [ ]:
def plot_candle_stick(df, title):
  fig = make_subplots(rows=2, cols=1,vertical_spacing=0.01, shared_xaxes=True, row_heights=[0.8,0.2])
  fig.append_trace(go.Candlestick(x=df['Date'],
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'],
                  name= "Price action"
                  ), row=1, col=1)

  fig.append_trace(go.Scatter(
      x=df['Date'],
      y=df['Volume'],
      name="Volume"
  ), row=2, col=1)

  fig.update_layout(xaxis_rangeslider_visible=False,title_text = title)
  fig['layout']['xaxis2']['title']='Date (From merger announcement)'
  fig['layout']['yaxis']['title']='Price($)'
  fig['layout']['yaxis2']['title']='Volume'

  fig.show()


In [ ]:
winners = ['SHLL', 'VTIQ', 'SPAQ', 'FMCI']
for ticker in winners:
  df = get_symbol_df(symbol=ticker, days = 30)
  plot_candle_stick(df, ticker+' price Volume chart')

**Inference**

All winners had one thing in common i.e. they had more than 10 million volume on the day of announcement.

## **Section 5: Top sectors by Volume and price action**

In this section, top 20% sectors are identified in terms of price and volume action

In [ ]:
#Find Average volume and price for each tickers
average_volume=[]
average_price=[]
for ticker in spac_tickers_list['spac_symbol']:
  ticker_data = get_symbol_df(symbol=ticker, days=300)
  average_volume.append(mean(ticker_data['Volume'].values.tolist()))
  average_price.append(mean(ticker_data['Close'].values.tolist()))

spac_tickers_list['average_volume'] = average_volume
spac_tickers_list['average_price'] = average_price

spac_tickers_list.head(5)

,spac_symbol,merge_symbol,name,news_or_rumor_date,price_before_news,merge_date,spac_market_cap,trust_value_dollars,merger_company_value_billions,spac_target_sector,merging_company_sector,average_volume,average_price
0,GPAQ,HOFV,Gordon Pointe Acquisition Corp,5/26/2020,11.00,7/1/2020,31060000,10.81,0.4935,FinTech,Entertainment,4.494378e+05,11.588808
1,ARYA,IMTX,ARYA Sciences Acquisition Corp,3/17/2020,10.58,7/1/2020,148698960,10.34,1.4000,Healthcare,Biotechnology,2.179366e+05,11.811703
2,PAAC,LGHL,Proficient Alpha Acquisition Corp,3/10/2020,10.11,6/16/2020,4586096,10.18,0.1250,FinTech,Asset Management,7.188791e+04,9.648397
3,NEBU,LPRO,Nebula Acquisition Corp,5/18/2020,10.30,6/10/2020,250000000,10.26,1.8000,Technology,Fintech,9.784526e+05,10.577813
4,VTIQ,NKLA,VectoIQ Acquisition Corp,3/3/2020,11.64,6/3/2020,200000000,10.33,3.5000,Automotive,EV,4.217753e+06,15.649453


In [ ]:
#Get average volume and price by sector 
sector_volume = spac_tickers_list.groupby('merging_company_sector')['average_volume'].mean().sort_values(ascending=True).reset_index().values.tolist()
sector_price = spac_tickers_list.groupby('merging_company_sector')['average_price'].mean().sort_values(ascending=True).reset_index().values.tolist()
print(sector_volume)
print(sector_price)

[['Information Technology', 1353.4350282485875], ['Pharma', 11974.915254237289], ['FinTech', 12359.139784946237], ['Mexico', 15060.013071895424], ['RealEstate', 56830.0], ['Engineering and Construction', 70654.80769230769], ['Asset Management', 71887.91176470589], ['Mining', 128835.85263157895], ['Insurance', 177815.9814814815], ['Biotechnology', 217936.63513513515], ['Marketing Technology', 245217.01754385966], ['Entertainment', 449437.8461538461], ['Cannabis', 500848.3870967742], ['Renewable Energy', 590075.8928571428], ['Fitness', 592087.5], ['Security', 629217.4418604651], ['Fintech', 978452.5625], ['Consumer', 1004185.0], ['Restaurants', 1015953.8461538461], ['Food Industry', 2255333.219334245], ['Gambling', 3176079.4176706825], ['Healthcare', 11391680.0], ['EV', 15387247.576155463]]
[['Asset Management', 9.64839705882353], ['Information Technology', 9.845180790960452], ['Mexico', 10.042699346405229], ['FinTech', 10.067225806451614], ['Engineering and Construction', 10.07455769230

In [ ]:
#Plot average volume by sector graph
columns = list(zip(*sector_volume)) #transpose rows to columns
color=np.array(['rgb(255,255,255)']*len(columns[0]))
max = len(columns[1])
color[-round(0.2*max):] = 'rgb(0,153,76)' #top 20%
color[-round(0.6*max):-round(0.2*max)] = 'rgb(255,153,51)'
color[:-round(0.6*max)] = 'rgb(255,0,0)'
fig = go.Figure(go.Bar(
            x=columns[1],
            y=columns[0],
            orientation='h',
            marker=dict(color=color.tolist())))

fig.update_layout(title_text = 'Average volume by Sectors',
                  annotations=get_annotations('Volume', ''))
fig.show()

In [ ]:
#Plot average price by sector graph
columns = list(zip(*sector_price)) #transpose rows to columns
fig = go.Figure(go.Bar(
            x=columns[1],
            y=columns[0],
            orientation='h',
            marker=dict(color=color.tolist())))

fig.update_layout(title_text = 'Average Price($) by Sectors',
                  annotations=get_annotations('Price($)', ''))
fig.show()

**Inference**

1. Sectors with top 20% trading volume are Ev, Healthcare, Gambling, Food and Restaurants in decending order.
2. Sectors with top 20% price actions are Ev, Restaurants, Food Industry, Gambling and Consumer in decending order.
3. Eventhough healthcare sector had high average volume, it didn't correlate to price action

## **Section 6: Merging Company Size and price action**

In [ ]:
spac_tickers_list.head(5)

fig = go.Figure()
fig.add_scatter(x=spac_tickers_list['merger_company_value_billions']*10**9, 
                y=spac_tickers_list['average_price'], 
                hovertemplate="Volume: %{x}<br> Average price:%{y}$",
                mode='markers',
                marker_size=spac_tickers_list['average_price'])

fig.update_layout(title_text = 'Merger company Value vs Average price',
                  annotations=get_annotations('Merger Value', 'Average price in $'))

fig.show()

**Inference**:

Company value more than 1 Billion dollars has better returns and better chance of success

## **Section 7: Conclusion**

The most likely winning stocks are those merging with following sectors in the order of high to lower priority
  1. EV
  2. Food Industry
  3. Restaurants (There could be data bias in this as we see increase in COVID cases, this might not be good for near term)
  4. Gambling/Gaming and
  5. Consumers

Also, If the stocks from one of the above sectors had 10 million or more volume on the first day of announcement and the merging company is valued above 1 billion dollars, then its a definite pick for winners.